In [28]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [31]:
data = pd.read_csv('ceylontoday/ceylon_today_sunday_urls.csv')
urls = data['url'].tolist()

FileNotFoundError: [Errno 2] No such file or directory: 'ceylontoday/ceylon_today_sunday_urls.csv'

In [25]:
scrape_url('https://ceylontoday.lk/2024/02/14/ports-ministry-says-harins-statement-not-true/')

{'url': 'https://ceylontoday.lk/2024/02/14/ports-ministry-says-harins-statement-not-true/',
 'title': 'Ports Ministry says Harin’s statement ‘NOT TRUE’',
 'author': 'By Sulochana Ramiah Mohan',
 'comments': '0',
 'Date': '2024-02-13T20:36:00+00:00',
 'Content': 'The Ministry of Ports, Shipping and Aviation has categorically stated that there is no truth to the claims made by Minister of Tourism Harin Fernando, during the Mumbai tourism roadshow, that an Indian investor might take control of the three main airports in Sri Lanka, including the BIA. The Tourism Minister, speaking at the Outbound Travel Mart (OTM), India’s leading and only travel show focused on outbound travel, said, “I will break the news that Indian companies are taking our three airports which is going to be a phenomenal decision to really help us. The fuel tanks are taken by the Indian companies, the solar power projects are also taken by the Indians.” Further, the Minister of Tourism also told a Hindu Businessline jo

In [24]:
def scrape_url(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Extract the title
    title_tag = soup.find('h1', class_='entry-title')
    title = title_tag.text if title_tag else 'Title not found'

    # Extract the author and content
    article_tag = soup.find('article')
    if article_tag:
        p_tags = article_tag.find_all('p')
        #if p_tags and p_tags[0].text.startswith('By '):
        if p_tags and (p_tags[0].text.startswith('By ') or p_tags[0].text.startswith('BY ')):    
            author = p_tags[0].text  # Author is extracted from the first <p> tag
            content = ' '.join(p.text for p in p_tags[1:])  # Exclude the first <p> tag from content
        else:
            author = 'Author not found'
            content = ' '.join(p.text for p in p_tags)  # Include all <p> tags in content
    else:
        content = 'Content not found'
        author = 'Author not found'

    # Extract the publication date
    date_tag = soup.find('meta', {'property': 'article:published_time'})
    date = date_tag['content'] if date_tag else 'Date not found'

    # Extract the number of comments
    comments_tag = soup.find('div', class_='td-post-comments')
    comments = comments_tag.find('a').text if comments_tag else 'Comments not found'


    return {
        'url': url,
        'title': title,
        'author': author,#.strip('author: '),
        'comments': comments,#.strip(' Comments'),
        'Date': date,
        'Content': content
    }

In [22]:
# Main function to scrape all URLs
def main():
    results = []
    for url in urls:
        try:
            result = scrape_url(url)
            results.append(result)
        except Exception as e:
            print(f"Error scraping {url}: {e}")

    #print(results)
    # Print the results
    #for result in results:
    #    print(result)
    
    # Convertendo os resultados para um DataFrame do Pandas
    df = pd.DataFrame(results)

    # Exportando o DataFrame para um arquivo CSV
    df.to_csv('sample_results_ceylon_today.csv', index=False, encoding='utf-8')

    #print("Exportação para CSV concluída com Pandas.")

if __name__ == "__main__":
    main()